# K-factor calibration

In [1]:
# setup
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import openmatrix as omx
import os

In [2]:
# file paths
projdir = r"C:\Users\michael.mccarthy\Resource Systems Group, Inc\Projects - 2024-Model & RTP Update-240130\3_Model"
survey_path = 'SCOG_HTS_trips_toNewZones_Autos.csv'
model_omx = os.path.join(projdir,'Tasks','6_Calibration','modExport.omx')
survey_omx = os.path.join(projdir,'Tasks','6_Calibration','SCOG_HTS_trips.omx')
replica_omx = os.path.join(projdir,'Tasks','6_Calibration','SCOG_Replica_trips.omx')
zonal_omx = os.path.join(projdir,'Tasks','6_Calibration','kfactor_zone.omx')

In [3]:
def omxtoDataframe(thismatrix,indexmap,corestr):
    df = pd.DataFrame(thismatrix, columns=indexmap, index=indexmap).reset_index().melt(id_vars='index').rename(columns = {'index':'origin', 'variable':'destination', 'value':corestr})
    return df

def district_kfac(ratio_mtx,district_map,core,limit_low,limit_high,quant):
    # limit/adjust
    ratio_mtx[ratio_mtx > limit_high] = limit_high
    ratio_mtx[ratio_mtx == 0] = np.nan
    ratio_mtx[ratio_mtx < limit_low] = limit_low
    # group by district
    mat_df = omxtoDataframe(ratio_mtx,district_map,core)
    mat_df_75 = mat_df.groupby(['origin','destination'])[core].quantile(quant).reset_index()
    # keep externals at 1.0
    mat_df_75.loc[(mat_df_75['origin'] == 99) | (mat_df_75['destination'] == 99),core] = 1
    # then disaggregate back into zonal matrix
    mat_k = mat_df.merge(mat_df_75, how='left', on=['origin','destination'])
    return np.array(mat_k[core+'_y']).reshape(372,372)

In [4]:
# load matrices
model_mtx = omx.open_file(model_omx)
survey_mtx = omx.open_file(survey_omx)
replica_mtx = omx.open_file(replica_omx)


In [5]:
# TAZ mapping
tazs = np.array(list(model_mtx.mapping('NO').keys()))

In [6]:
districts_path = os.path.join(projdir,'Data','Passive Data 2019','TAZ2022_Districts.csv')
districts = pd.read_csv(districts_path)
districts = districts[['NO','zone_id']]
districts = districts.rename(columns={'zone_id':'District'})
dist_map = np.array(list(districts['District']))

In [7]:
# adjust to model trips total
survey_hbw_scale = np.array(model_mtx['21']).sum() / np.array(survey_mtx['HBW']).sum()
survey_hbo_scale = np.array(model_mtx['22']).sum() / np.array(survey_mtx['HBO']).sum()
survey_nhb_scale = np.array(model_mtx['26']).sum() / np.array(survey_mtx['NHB']).sum()

replica_hbw_scale = np.array(model_mtx['21']).sum() / np.array(replica_mtx['HBW']).sum()
replica_hbo_scale = np.array(model_mtx['22']).sum() / np.array(replica_mtx['HBO']).sum()
replica_nhb_scale = np.array(model_mtx['26']).sum() / np.array(replica_mtx['NHB']).sum()

# blended observed data
obs_hbw = np.array(survey_mtx['HBW'] * survey_hbw_scale) * 0.75 + np.array(replica_mtx['HBW'] * replica_hbw_scale) * 0.25
obs_hbo = np.array(survey_mtx['HBO'] * survey_hbo_scale) * 0.75 + np.array(replica_mtx['HBO'] * replica_hbo_scale) * 0.25
obs_nhb = np.array(survey_mtx['NHB'] * survey_nhb_scale) * 0.75 + np.array(replica_mtx['NHB'] * replica_nhb_scale) * 0.25

In [8]:
# HBW observed / model
mod_hbw = np.array(model_mtx['21'])
hbw_vs_obs = obs_hbw / mod_hbw

# HBO observed / model
mod_hbo = np.array(model_mtx['22'])
hbo_vs_obs = obs_hbo / mod_hbo

# NHB observed / model
mod_nhb = np.array(model_mtx['26'])
nhb_vs_obs = obs_nhb / mod_nhb

C:\Users\michael.mccarthy\AppData\Local\Temp\ipykernel_23828\989469914.py:3: RuntimeWarning: divide by zero encountered in divide
  hbw_vs_obs = obs_hbw / mod_hbw
C:\Users\michael.mccarthy\AppData\Local\Temp\ipykernel_23828\989469914.py:3: RuntimeWarning: invalid value encountered in divide
  hbw_vs_obs = obs_hbw / mod_hbw
C:\Users\michael.mccarthy\AppData\Local\Temp\ipykernel_23828\989469914.py:7: RuntimeWarning: divide by zero encountered in divide
  hbo_vs_obs = obs_hbo / mod_hbo
C:\Users\michael.mccarthy\AppData\Local\Temp\ipykernel_23828\989469914.py:7: RuntimeWarning: invalid value encountered in divide
  hbo_vs_obs = obs_hbo / mod_hbo
C:\Users\michael.mccarthy\AppData\Local\Temp\ipykernel_23828\989469914.py:11: RuntimeWarning: divide by zero encountered in divide
  nhb_vs_obs = obs_nhb / mod_nhb
C:\Users\michael.mccarthy\AppData\Local\Temp\ipykernel_23828\989469914.py:11: RuntimeWarning: invalid value encountered in divide
  nhb_vs_obs = obs_nhb / mod_nhb


In [9]:
# create omx to hold zonal result
zonal = omx.open_file(zonal_omx,'w')
zonal.create_mapping('NO', tazs) # add taz mapping

/lookup/NO (Array(372,)) np.str_('')
  atom := UInt32Atom(shape=(), dflt=np.uint32(0))
  maindim := 0
  flavor := 'numpy'
  byteorder := 'little'
  chunkshape := None

In [10]:
# limits initially 0.8 to 1.2
zonal['HBW_K'] = district_kfac(hbw_vs_obs,dist_map,'HBW',0.5,1.5,0.75)
zonal['HBO_K'] = district_kfac(hbo_vs_obs,dist_map,'HBO',0.5,1.5,0.75)
zonal['NHB_K'] = district_kfac(nhb_vs_obs,dist_map,'NHB',0.5,1.5,0.75)

In [11]:
# close files
model_mtx.close()
survey_mtx.close()
replica_mtx.close()
zonal.close()